In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
pd.set_option('display.float_format', '{:.2f}'.format)
np.random.seed(42)

# Chargement des données
def load_data():
    data_dir = Path('data')
    try:
        df_sales = pd.read_csv(data_dir/'ventes_enhanced.csv', parse_dates=['Date'])
        df_weather = pd.read_csv(data_dir/'meteo_locale.csv', parse_dates=['Date'])
        df_products = pd.read_csv(data_dir/'produits.csv')
        df_staff = pd.read_csv(data_dir/'planning_equipes.csv', parse_dates=['Date'])
        df_marketing = pd.read_csv(data_dir/'campagnes_marketing.csv', encoding='latin1', parse_dates=['Date_début', 'Date_fin'])
        return {
            'sales': df_sales,
            'weather': df_weather,
            'products': df_products,
            'staff': df_staff,
            'marketing': df_marketing
        }
    except Exception as e:
        print(f"❌ Erreur lors du chargement des données: {str(e)}")
        return None

# Préparation des données
def prepare_data(df_sales, df_weather, df_marketing):
    df_daily = df_sales.groupby('Date').agg({'CA': 'sum', 'Magasin': 'nunique'}).reset_index()
    df_weather_daily = df_weather.groupby('Date')['Température'].mean().reset_index()
    df_weather_daily.rename(columns={'Température': 'temperature'}, inplace=True)
    df_merged = pd.merge(df_daily, df_weather_daily, on='Date')
    df_merged['promo_active'] = df_merged['Date'].apply(
        lambda d: any((d >= row['Date_début']) & (d <= row['Date_fin']) for _, row in df_marketing.iterrows())
    ).astype(int)
    return df_merged

# Création du modèle Prophet
def create_model():
    model = Prophet(weekly_seasonality=True, yearly_seasonality=False,
                    changepoint_prior_scale=0.3, seasonality_mode='multiplicative')
    model.add_regressor('temperature')
    model.add_regressor('promo_active')
    model.add_country_holidays(country_name='FR')
    return model

# Entraînement + validation + MAPE
def train_and_validate(df):
    split_date = df['Date'].quantile(0.8)
    train = df[df['Date'] <= split_date]
    test = df[df['Date'] > split_date]
    train_prophet = train.rename(columns={'Date': 'ds', 'CA': 'y'})
    model = create_model()
    model.fit(train_prophet)
    future = model.make_future_dataframe(periods=len(test))
    future = future.merge(df[['Date', 'temperature', 'promo_active']], left_on='ds', right_on='Date', how='left')
    forecast = model.predict(future)
    from sklearn.metrics import mean_absolute_percentage_error
    mape = mean_absolute_percentage_error(test['CA'], forecast.tail(len(test))['yhat'])
    return model, forecast, mape

# Affichage
def plot_forecast(model, forecast):
    fig1 = model.plot(forecast)
    plt.title("Prévisions de chiffre d'affaires")
    plt.show()
    fig2 = model.plot_components(forecast)
    plt.show()

# Exécution complète
data = load_data()
if data:
    df = prepare_data(data['sales'], data['weather'], data['marketing'])
    model, forecast, mape = train_and_validate(df)
    print(f"✅ MAPE: {mape:.1%} (objectif : < 15%)")
    plot_forecast(model, forecast)


AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json
import holidays
import os
from pathlib import Path

# Configuration initiale
pd.set_option('display.float_format', '{:.2f}'.format)
np.random.seed(42)

# 1. Chargement des données - Version améliorée
def load_data():
    """Charge tous les fichiers CSV nécessaires avec gestion des erreurs"""
    data_dir = Path('data')
    try:
        df_sales = pd.read_csv(data_dir/'ventes_enhanced.csv', parse_dates=['Date'])
        df_weather = pd.read_csv(data_dir/'meteo_locale.csv', parse_dates=['Date'])
        df_products = pd.read_csv(data_dir/'produits.csv')
        df_staff = pd.read_csv(data_dir/'planning_equipes.csv', parse_dates=['Date'])
        
        return {
            'sales': df_sales,
            'weather': df_weather,
            'products': df_products,
            'staff': df_staff
        }
    except Exception as e:
        print(f"❌ Erreur lors du chargement des données: {str(e)}")
        return None

# 2. Nettoyage des données - Version robuste
def clean_data(df_sales, df_weather):
    """Nettoie et fusionne les données avec vérifications"""
    try:
        # Vérification des entrées
        if df_sales.empty or df_weather.empty:
            raise ValueError("DataFrames d'entrée vides")
            
        # Gestion des outliers avec seuil dynamique
        q_low = df_sales['CA'].quantile(0.005)
        q_high = df_sales['CA'].quantile(0.995)
        df_sales = df_sales[(df_sales['CA'] > q_low) & (df_sales['CA'] < q_high)]
        
        # Détection des jours fermés
        daily_sales = df_sales.groupby('Date')['CA'].sum()
        closed_days = daily_sales[daily_sales == 0].index
        df_sales = df_sales[~df_sales['Date'].isin(closed_days)]
        
        # Fusion avec vérification des clés
        df_merged = pd.merge(
            df_sales.groupby(['Date', 'Magasin'])['CA'].sum().reset_index(),
            df_weather,
            left_on=['Date', 'Magasin'],
            right_on=['Date', 'Ville'],
            how='left'
        ).dropna(subset=['CA'])
        
        # Jours fériés avec gestion des années
        years = df_merged['Date'].dt.year.unique()
        fr_holidays = holidays.France(years=years)
        df_merged['is_holiday'] = df_merged['Date'].apply(lambda x: x in fr_holidays)
        
        return df_merged
        
    except Exception as e:
        print(f"❌ Erreur lors du nettoyage: {str(e)}")
        return None

# 3. Feature Engineering - Optimisé
def create_features(df):
    """Crée des variables supplémentaires avec gestion des erreurs"""
    try:
        # Vérification des colonnes nécessaires
        if 'Date' not in df.columns or 'CA' not in df.columns:
            raise ValueError("Colonnes manquantes")
            
        # Variables temporelles
        df['day_of_week'] = df['Date'].dt.dayofweek
        df['month'] = df['Date'].dt.month
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        
        # Features temporelles avancées
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)
        
        # Lag features avec fenêtre glissante
        for lag in [1, 7, 14]:
            df[f'CA_lag{lag}'] = df.groupby('Magasin')['CA'].shift(lag)
            
        # Rolling statistics
        for window in [7, 14, 28]:
            df[f'CA_rolling{window}_mean'] = df.groupby('Magasin')['CA'].transform(
                lambda x: x.rolling(window, min_periods=1).mean()
            )
            df[f'CA_rolling{window}_std'] = df.groupby('Magasin')['CA'].transform(
                lambda x: x.rolling(window, min_periods=1).std()
            )
        
        return df.dropna().reset_index(drop=True)
        
    except Exception as e:
        print(f"❌ Erreur dans le feature engineering: {str(e)}")
        return None

# 4. Préparation pour Prophet - Adaptatif
def prepare_for_prophet(df, magasin_id=None):
    """Prépare les données pour Prophet avec sélection de magasin"""
    try:
        if magasin_id:
            df = df[df['Magasin'] == magasin_id].copy()
            
        # Sélection des colonnes pertinentes
        keep_cols = ['Date', 'CA', 'Température', 'is_weekend', 'is_holiday']
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        keep_cols += [col for col in numeric_cols if col not in keep_cols and 'CA_' in col]
        
        prophet_df = df[keep_cols].rename(columns={'Date': 'ds', 'CA': 'y'})
        
        # Normalisation des noms de colonnes
        prophet_df.columns = [col.lower().replace('é', 'e') for col in prophet_df.columns]
        
        return prophet_df.dropna()
        
    except Exception as e:
        print(f"❌ Erreur de préparation Prophet: {str(e)}")
        return None

# 5. Modélisation Prophet - Version professionnelle
class ProphetModel:
    def __init__(self, params=None):
        self.params = params or {
            'daily_seasonality': False,
            'weekly_seasonality': True,
            'yearly_seasonality': True,
            'changepoint_prior_scale': 0.05,
            'seasonality_mode': 'multiplicative'
        }
        self.model = None
        
    def train(self, df):
        """Entraîne le modèle avec gestion complète des erreurs"""
        try:
            # Validation des données
            if df is None or df.empty:
                raise ValueError("Données d'entrée invalides")
                
            required_cols = {'ds', 'y'}
            if not required_cols.issubset(df.columns):
                raise ValueError(f"Colonnes requises manquantes: {required_cols - set(df.columns)}")
            
            # Initialisation du modèle
            self.model = Prophet(**self.params)
            
            # Ajout des regresseurs dynamiques
            regressors = [col for col in df.columns if col not in required_cols]
            for reg in regressors:
                try:
                    self.model.add_regressor(reg)
                except Exception as e:
                    print(f"⚠️ Regresseur {reg} ignoré: {str(e)}")
            
            # Entraînement
            self.model.fit(df)
            print("✅ Modèle entraîné avec succès")
            return True
            
        except Exception as e:
            print(f"❌ Échec de l'entraînement: {str(e)}")
            self.model = None
            return False
    
    def validate(self, df, initial='60 days', period='15 days', horizon='30 days', mape_threshold=0.15):
        """Validation croisée"""
        if self.model is None:
            print("❌ Modèle non entraîné")
            return False
            
        try:
            df_cv = cross_validation(
                self.model,
                initial=initial,
                period=period,
                horizon=horizon
            )
            
            metrics = performance_metrics(df_cv)
            mape = metrics['mape'].mean()
            print(f"📊 Performance MAPE: {mape:.2%}")
            
            return mape <= mape_threshold
            
        except Exception as e:
            print(f"❌ Échec de validation: {str(e)}")
            return False
    
    def save_model(self, path='models/prophet_model.json'):
        """Sauvegarde le modèle"""
        try:
            os.makedirs(os.path.dirname(path), exist_ok=True)
            with open(path, 'w') as f:
                f.write(model_to_json(self.model))
            print(f"✅ Modèle sauvegardé: {path}")
            return True
        except Exception as e:
            print(f"❌ Échec de sauvegarde: {str(e)}")
            return False

# Pipeline complet avec journalisation
def main():
    print("\n" + "="*50)
    print("🚀 Démarrage du pipeline de prévision")
    print("="*50 + "\n")
    
    # 1. Chargement des données
    print("🔍 Étape 1/5 - Chargement des données...")
    data = load_data()
    if data is None:
        return

    # 2. Nettoyage et fusion
    print("\n🧹 Étape 2/5 - Nettoyage et fusion des données...")
    df_clean = clean_data(data['sales'], data['weather'])
    if df_clean is None:
        return
    print(f"📊 Données après nettoyage: {df_clean.shape}")

    # 3. Feature engineering
    print("\n⚙️ Étape 3/5 - Création des features...")
    df_features = create_features(df_clean)
    if df_features is None:
        return
    print(f"✨ Features ajoutées: {len(df_features.columns)} au total")

    # 4. Préparation Prophet
    print("\n📊 Étape 4/5 - Préparation pour Prophet...")
    prophet_data = prepare_for_prophet(df_features, magasin_id='Magasin_1')
    if prophet_data is None:
        return
    print(f"📅 Période couverte: {prophet_data['ds'].min().date()} au {prophet_data['ds'].max().date()}")

    # 5. Modélisation
    print("\n🤖 Étape 5/5 - Modélisation Prophet...")
    model = ProphetModel()
    if model.train(prophet_data) and model.validate(prophet_data):
        model.save_model()
        print("\n🎉 Pipeline exécuté avec succès!")
    else:
        print("\n❌ Le pipeline a rencontré des problèmes")

if __name__ == "__main__":
    main()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




🚀 Démarrage du pipeline de prévision

🔍 Étape 1/5 - Chargement des données...

🧹 Étape 2/5 - Nettoyage et fusion des données...
📊 Données après nettoyage: (455, 7)

⚙️ Étape 3/5 - Création des features...
✨ Features ajoutées: 21 au total

📊 Étape 4/5 - Préparation pour Prophet...
📅 Période couverte: NaT au NaT

🤖 Étape 5/5 - Modélisation Prophet...
❌ Échec de l'entraînement: Données d'entrée invalides

❌ Le pipeline a rencontré des problèmes
